In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing as mp
import logging
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns', 700)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all columns
pd.set_option('display.float_format', '{:.5f}'.format)
WINDOW = 24

In [2]:
path = f'Data/DF_Combined_2025_1h.csv'
df = pd.read_csv(path)
df['open_time'] = pd.to_datetime(df['open_time'])
df['open_time'] = pd.to_datetime(df['open_time']).dt.tz_localize(None)
print(f'DF shape - {df.shape}')


def get_coins(df: pd.DataFrame) -> list:
    cols = df.columns.tolist()
    coins = []
    for col in cols:
        if col.endswith('_open'):
            coin = col[:-5]
            coins.append(coin)
    return coins


df.head()

DF shape - (6785, 253)


,open_time,BTCUSDT_open,BTCUSDT_high,BTCUSDT_low,BTCUSDT_close,BTCUSDT_volume,BTCUSDT_RET,AAVEUSDT_open,AAVEUSDT_high,AAVEUSDT_low,AAVEUSDT_close,AAVEUSDT_volume,AAVEUSDT_RET,ADAUSDT_open,ADAUSDT_high,ADAUSDT_low,ADAUSDT_close,ADAUSDT_volume,ADAUSDT_RET,ALGOUSDT_open,ALGOUSDT_high,ALGOUSDT_low,ALGOUSDT_close,ALGOUSDT_volume,ALGOUSDT_RET,APTUSDT_open,APTUSDT_high,APTUSDT_low,APTUSDT_close,APTUSDT_volume,APTUSDT_RET,ARBUSDT_open,ARBUSDT_high,ARBUSDT_low,ARBUSDT_close,ARBUSDT_volume,ARBUSDT_RET,ATOMUSDT_open,ATOMUSDT_high,ATOMUSDT_low,ATOMUSDT_close,ATOMUSDT_volume,ATOMUSDT_RET,AVAXUSDT_open,AVAXUSDT_high,AVAXUSDT_low,AVAXUSDT_close,AVAXUSDT_volume,AVAXUSDT_RET,BCHUSDT_open,BCHUSDT_high,BCHUSDT_low,BCHUSDT_close,BCHUSDT_volume,BCHUSDT_RET,BNBUSDT_open,BNBUSDT_high,BNBUSDT_low,BNBUSDT_close,BNBUSDT_volume,BNBUSDT_RET,DOGEUSDT_open,DOGEUSDT_high,DOGEUSDT_low,DOGEUSDT_close,DOGEUSDT_volume,DOGEUSDT_RET,DOTUSDT_open,DOTUSDT_high,DOTUSDT_low,DOTUSDT_close,DOTUSDT_volume,DOTUSDT_RET,ENAUSDT_open,ENAUSDT_high,ENAUSDT_low,ENAUSDT_close,ENAUSDT_volume,ENAUSDT_RET,ENSUSDT_open,ENSUSDT_high,ENSUSDT_low,ENSUSDT_close,ENSUSDT_volume,ENSUSDT_RET,ETCUSDT_open,ETCUSDT_high,ETCUSDT_low,ETCUSDT_close,ETCUSDT_volume,ETCUSDT_RET,ETHUSDT_open,ETHUSDT_high,ETHUSDT_low,ETHUSDT_close,ETHUSDT_volume,ETHUSDT_RET,FETUSDT_open,FETUSDT_high,FETUSDT_low,FETUSDT_close,FETUSDT_volume,FETUSDT_RET,FILUSDT_open,FILUSDT_high,FILUSDT_low,FILUSDT_close,FILUSDT_volume,FILUSDT_RET,HBARUSDT_open,HBARUSDT_high,HBARUSDT_low,HBARUSDT_close,HBARUSDT_volume,HBARUSDT_RET,ICPUSDT_open,ICPUSDT_high,ICPUSDT_low,ICPUSDT_close,ICPUSDT_volume,ICPUSDT_RET,JUPUSDT_open,JUPUSDT_high,JUPUSDT_low,JUPUSDT_close,JUPUSDT_volume,JUPUSDT_RET,LINKUSDT_open,LINKUSDT_high,LINKUSDT_low,LINKUSDT_close,LINKUSDT_volume,LINKUSDT_RET,LTCUSDT_open,LTCUSDT_high,LTCUSDT_low,LTCUSDT_close,LTCUSDT_volume,LTCUSDT_RET,NEARUSDT_open,NEARUSDT_high,NEARUSDT_low,NEARUSDT_close,NEARUSDT_volume,NEARUSDT_RET,ONDOUSDT_open,ONDOUSDT_high,ONDOUSDT_low,ONDOUSDT_close,ONDOUSDT_volume,ONDOUSDT_RET,OPUSDT_open,OPUSDT_high,OPUSDT_low,OPUSDT_close,OPUSDT_volume,OPUSDT_RET,PENGUUSDT_open,PENGUUSDT_high,PENGUUSDT_low,PENGUUSDT_close,PENGUUSDT_volume,PENGUUSDT_RET,POLUSDT_open,POLUSDT_high,POLUSDT_low,POLUSDT_close,POLUSDT_volume,POLUSDT_RET,RAYUSDT_open,RAYUSDT_high,RAYUSDT_low,RAYUSDT_close,RAYUSDT_volume,RAYUSDT_RET,RENDERUSDT_open,RENDERUSDT_high,RENDERUSDT_low,RENDERUSDT_close,RENDERUSDT_volume,RENDERUSDT_RET,SEIUSDT_open,SEIUSDT_high,SEIUSDT_low,SEIUSDT_close,SEIUSDT_volume,SEIUSDT_RET,SOLUSDT_open,SOLUSDT_high,SOLUSDT_low,SOLUSDT_close,SOLUSDT_volume,SOLUSDT_RET,SUIUSDT_open,SUIUSDT_high,SUIUSDT_low,SUIUSDT_close,SUIUSDT_volume,SUIUSDT_RET,TAOUSDT_open,TAOUSDT_high,TAOUSDT_low,TAOUSDT_close,TAOUSDT_volume,TAOUSDT_RET,TONUSDT_open,TONUSDT_high,TONUSDT_low,TONUSDT_close,TONUSDT_volume,TONUSDT_RET,TRXUSDT_open,TRXUSDT_high,TRXUSDT_low,TRXUSDT_close,TRXUSDT_volume,TRXUSDT_RET,UNIUSDT_open,UNIUSDT_high,UNIUSDT_low,UNIUSDT_close,UNIUSDT_volume,UNIUSDT_RET,VETUSDT_open,VETUSDT_high,VETUSDT_low,VETUSDT_close,VETUSDT_volume,VETUSDT_RET,WLDUSDT_open,WLDUSDT_high,WLDUSDT_low,WLDUSDT_close,WLDUSDT_volume,WLDUSDT_RET,XLMUSDT_open,XLMUSDT_high,XLMUSDT_low,XLMUSDT_close,XLMUSDT_volume,XLMUSDT_RET,XRPUSDT_open,XRPUSDT_high,XRPUSDT_low,XRPUSDT_close,XRPUSDT_volume,XRPUSDT_RET,ZECUSDT_open,ZECUSDT_high,ZECUSDT_low,ZECUSDT_close,ZECUSDT_volume,ZECUSDT_RET
0,2024-12-31 23:00:00,93469.10000,93736.90000,93356.60000,93548.90000,3036.94600,0.00085,309.59000,311.66000,307.85000,308.70000,16569.70000,-0.00287,0.84400,0.84640,0.84170,0.84470,10080834,0.00071,0.33340,0.33610,0.33290,0.33440,6425491.90000,0.00300,8.70260,8.74410,8.67540,8.71210,399404.20000,0.00105,0.72200,0.72400,0.71980,0.72180,2892544.20000,-0.00028,6.18900,6.21300,6.17600,6.18700,167108.60000,-0.00032,35.63100,35.77200,35.55400,35.68800,132257,0.00160,433.81000,435.66000,432.86000,434.36000,6274.99100,0.00127,700.84000,702.59000,699.11000,702.12000,22770.66000,0.00184,0.31593,

In [3]:
def wide_crypto_to_long(df, time_col="open_time", sort_index=True, set_multiindex=True):
    """
    Convert a wide-format crypto OHLCV/features DataFrame to long format:
      - Rows: date × coin
      - Columns: features (close, volume, RET, *_MA*, *_TREND, etc.)

    Parameters
    ----------
    df : pd.DataFrame
        Wide DataFrame with columns like COIN_feature (e.g., BTCUSDT_close).
    time_col : str
        Name of the datetime/time column.
    sort_index : bool
        Whether to sort the resulting MultiIndex columns.
    set_multiindex : bool
        If True, return DataFrame indexed by ['coin','date'].

    Returns
    -------
    pd.DataFrame
        Long-format DataFrame.
    """
    out = df.copy()
    out[time_col] = pd.to_datetime(out[time_col], errors="coerce")

    # Identify all coin_feature columns
    value_cols = [c for c in out.columns if c != time_col and "_" in c]

    # Build MultiIndex on columns: level 0 = coin, level 1 = feature
    coins, feats = zip(*[c.split("_", 1) for c in value_cols])
    out_values = out[value_cols].copy()
    out_values.columns = pd.MultiIndex.from_arrays([coins, feats], names=["coin", "feature"])

    if sort_index:
        out_values = out_values.sort_index(axis=1)

    # Stack by coin → rows become (time, coin)
    tmp = out_values.assign(**{time_col: out[time_col].values}).set_index(time_col)
    try:
        stacked = tmp.stack(level="coin", future_stack=True)  # pandas ≥ 2.1
    except TypeError:
        stacked = tmp.stack(level="coin")  # fallback for older versions

    # Clean index names
    stacked.index = stacked.index.set_names(["date", "coin"])
    long_df = stacked.reset_index()  # columns: date, coin, <features...>

    if set_multiindex:
        return long_df.set_index(["coin", "date"]).sort_index()
    return long_df


In [4]:
long_df = wide_crypto_to_long(df)
long_df = long_df.dropna()
long_df = long_df.reset_index()
long_df.head()

feature,coin,date,RET,close,high,low,open,volume
0,AAVEUSDT,2024-12-31 23:00:00,-0.00287,308.70000,311.66000,307.85000,309.59000,16569.70000
1,AAVEUSDT,2025-01-01 00:00:00,0.01591,313.61000,313.72000,308.72000,308.73000,27473.90000
2,AAVEUSDT,2025-01-01 01:00:00,-0.00077,313.37000,314.28000,311.37000,313.61000,23172.20000
3,AAVEUSDT,2025-01-01 02:00:00,0.00377,314.55000,316.92000,313.21000,313.37000,26176.50000
4,AAVEUSDT,2025-01-01 03:00:00,-0.00741,312.22000,314.58000,311.81000,314.53000,13005.10000


In [5]:
long_df.to_csv('Data/DF_Combined_2025_1h_Processed.csv', index=False)